In [ ]:
from google.colab import drive
import os
import numpy as np
import librosa
from scipy import signal
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Path to the directory containing the audio files in Google Drive
drive_directory_path = '/content/drive/My Drive/WAV'

# Filter for WAV files in the directory
audio_files = [os.path.join(drive_directory_path, file) for file in os.listdir(drive_directory_path) if file.endswith('.wav')]

# Count the number of WAV files found
num_files = len(audio_files)
print(f"Number of WAV files found: {num_files}")

# Constants TAKEN FROM EXAMPLE IN BANDPASS FILTERING AND PREPROCESSING BY KAGGLE PROJECT
gSampleRate = 7000
upperCutoffFreq = 3000
cutoffFrequencies = [80, upperCutoffFreq]

## PREPROCESSING FEATURE EXTRACTION IN DATA DEVELOPMENT

# Load and preprocess audio files
def load_and_preprocess_files(audio_file_paths):


# Extracting features
spectrograms = [calculate_spectrogram(audio, gSampleRate)[0] for audio in audioBuffers]
mfccs = [calculate_mfcc(audio, gSampleRate) for audio in audioBuffers]
chromagrams = [calculate_chromagram(audio, gSampleRate) for audio in audioBuffers]

# Reshape the features if  to  the same number of frames (time )
min_frames = min(spec.shape[1] for spec in spectrograms)
spectrograms = [spec[:, :min_frames] for spec in spectrograms]
mfccs = [mfcc[:, :min_frames] for mfcc in mfccs]
chromagrams = [chroma[:, :min_frames] for chroma in chromagrams]

# Convert features to numpy arrays and transpose them to have frames as rows and features as columns
spectrograms_np = np.array([spec.T for spec in spectrograms])
mfccs_np = np.array([mfcc.T for mfcc in mfccs])
chromagrams_np = np.array([chroma.T for chroma in chromagrams])

#  features as one input array
combined_features = np.concatenate((spectrograms_np, mfccs_np, chromagrams_np), axis=2)
print(combined_features.shape)


# Build an RNN model NOT ROBUST ENOUGH (THIS IS INITIAL ITERATION)
model = Sequential()
model.add(LSTM(128, input_shape=(combined_features.shape[1], combined_features.shape[2])))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Reshape features for RNN input (adding the time dimension)
combined_features = np.reshape(combined_features, (combined_features.shape[0], combined_features.shape[1], combined_features.shape[2]))

# Train the model
model.fit(combined_features, labels, epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, LeakyReLU, BatchNormalization

# Features should already be prepared

model = Sequential()

# LSTM layers with increased units and stacking for deeper learning & complexity in pattern recognition
model.add(LSTM(256, input_shape=(combined_features.shape[1], combined_features.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.8))
model.add(LSTM(64))
model.add(Dropout(0.5))

# Additional dense layers for better pass through learning
model.add(Dense(128))
model.add(ReLU(alpha=0.2))
model.add(BatchNormalization())
model.add(Dropout(0.8))

model.add(Dense(64))
model.add(LeakyReLU(alpha=0.2))
model.add(BatchNormalization())
model.add(Dropout(0.8))

# Output layer with sigmoid activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Reshape features for RNN input (adding the time dimension)
combined_features = np.reshape(combined_features, (combined_features.shape[0], combined_features.shape[1], 1))

# Train the model
model.fit(combined_features, labels, epochs=350, batch_size=64, validation_split=0.2)


In [ ]:
# pruning  the model
def apply_pruning_to_model(model):
    pruning_schedule = tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.00, final_sparsity=0.50, begin_step=0, end_step=1000
    )
    pruning_callbacks = [
        tfmot.sparsity.keras.UpdatePruningStep(),
        tfmot.sparsity.keras.PruningSummaries(log_dir='/content/drive/My Drive/pruning_logs'),
    ]
    model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(model, pruning_schedule=pruning_schedule)
    model_for_pruning.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model_for_pruning

## NEW MODEL WILL BE QUANTIZED
pruned_model = apply_pruning_to_model(model)

# BALANCE THE WEIGHTS (KAGGLE SOURCE FOR AUDIO)
pruned_model.fit(combined_features, labels, epochs=5, batch_size=64, validation_split=0.2)

# Convert 2 a quantized version
converter = tf.lite.TFLiteConverter.from_keras_model(pruned_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_tflite_model = converter.convert()

# Save  quantized model to a file
with open('/content/drive/My Drive/quantized_audio_model_optimized.tflite', 'wb') as f:
    f.write(audio_model)
## SAVE AS A .H FILE

